In [2]:
# A Django dashboard showing real-time Wikidata links

# screenshot: https://acmi-gallery-5.s3.ap-southeast-2.amazonaws.com/acmi/wikidata-dashboard.png

In [ ]:
# Python/Django view

class WikidataView(View):
    """
    Shows statistics on the number of ACMI collection IDs in Wikidata.
    """

    template_name = 'collection/wikidata.html'

    def get(self, request, *args, **kwargs):
        """
        Render charts showing the number of Works and Creators IDs linked to and from Wikidata.
        """
        works = Work.objects.filter(
            unpublished=False,
        ).distinct().count()
        creators = Creator.objects.distinct().count()

        wikidata_client = Wikidata()
        xos = wikidata_client.get_xos_records_with_wikidata_ids()
        wikidata = wikidata_client.get_wikidata_entities_with_acmi_ids()

        # Chart data
        xos_works_percentage = round(((len(xos['works']) / works) * 100), 1)
        xos_creators_percentage = round(((len(xos['creators']) / creators) * 100), 1)
        wikidata_works_percentage = round(((len(wikidata['works']) / works) * 100), 1)
        wikidata_creators_percentage = round(((len(wikidata['creators']) / creators) * 100), 1)

        data = {
            'xosWorks': [xos_works_percentage, 100 - xos_works_percentage],
            'xosCreators': [xos_creators_percentage, 100 - xos_creators_percentage],
            'wikidataWorks': [wikidata_works_percentage, 100 - wikidata_works_percentage],
            'wikidataCreators': [wikidata_creators_percentage, 100 - wikidata_creators_percentage],
        }

        return render(
            request,
            self.template_name,
            {
                'works': works,
                'creators': creators,
                'xos': xos,
                'wikidata': wikidata,
                'data': data,
            },
        )

In [ ]:
# HTML template using https://www.chartjs.org

{% block page_content %}

  <article class="researchers">
    <h1>ACMI x Wikidata</h1>
    <hr>
    <h2>XOS</h2>
    <p>How many ACMI collection items have Wikidata IDs:</p>
    <div class="row">
      <div class="chart col-sm">
        <h3>{{ data.xosWorks.0 }}% of Works</h3>
        <canvas id="xosWorks"></canvas>
      </div>
      <div class="chart col-sm">
        <h3>{{ data.xosCreators.0 }}% of Creators</h3>
        <canvas id="xosCreators"></canvas>
      </div>
    </div>
    <h2>Wikidata</h2>
    <p>How many Wikidata entities have ACMI IDs vs total ACMI collection items:</p>
    <div class="row">
      <div class="chart col-sm">
        <h3>{{ data.wikidataWorks.0 }}% of Works</h3>
        <canvas id="wikidataWorks"></canvas>
      </div>
      <div class="chart col-sm">
        <h3>{{ data.wikidataCreators.0 }}% of Creators</h3>
        <canvas id="wikidataCreators"></canvas>
      </div>
    </div>
  </article>

{% endblock %}

{% block javascript %}
  <script type="text/javascript" src="{% static "js/libs/Chart.bundle.min.js.css" %}"></script>
  <script type="text/javascript">
    // Charts
    const xosWorks = document.getElementById('xosWorks');
    const xosWorksData = {
      labels: [
        'Works with Wikidata IDs',
        'No Wikidata IDs',
      ],
      datasets: [{
        label: 'XOS Works',
        data: [{{ xos.works|length }}, {{ works }}],
        backgroundColor: [
          'rgb(74, 192, 192)',
          'rgb(255, 99, 131)',
        ],
        borderColor: 'black',
        hoverOffset: 4
      }]
    };
    const xosWorksConfig = {
      type: 'doughnut',
      data: xosWorksData,
      overrides: {
        responsive: true,
        plugins: {
          legend: {
            position: 'bottom',
            display: false
          },
          title: {
            display: true,
            text: 'XOS Works'
          }
        }
      },
    };
    new Chart(xosWorks, xosWorksConfig);

    const xosCreators = document.getElementById('xosCreators');
    const xosCreatorsData = {
      labels: [
        'Creators with Wikidata IDs',
        'No Wikidata IDs',
      ],
      datasets: [{
        label: 'XOS Creators',
        data: [{{ xos.creators|length }}, {{ creators }}],
        backgroundColor: [
          'rgb(74, 192, 192)',
          'rgb(255, 99, 131)',
        ],
        borderColor: 'black',
        hoverOffset: 4
      }]
    };
    const xosCreatorsConfig = {
      type: 'doughnut',
      data: xosCreatorsData,
      options: {
        responsive: true,
        plugins: {
          legend: {
            position: 'bottom',
          },
          title: {
            display: true,
            text: 'XOS Creators',
          }
        }
      },
    };
    new Chart(xosCreators, xosCreatorsConfig);

    const wikidataWorks = document.getElementById('wikidataWorks');
    const wikidataWorksData = {
      labels: [
        'Works with ACMI IDs',
        'No ACMI IDs',
      ],
      datasets: [{
        label: 'XOS Works',
        data: [{{ wikidata.works|length }}, {{ works }}],
        backgroundColor: [
          'rgb(74, 192, 192)',
          'rgb(255, 99, 131)',
        ],
        borderColor: 'black',
        hoverOffset: 4
      }]
    };
    const wikidataWorksConfig = {
      type: 'doughnut',
      data: wikidataWorksData,
      options: {
        responsive: true,
        plugins: {
          legend: {
            position: 'bottom',
          },
          title: {
            display: true,
            text: 'Wikidata Works',
          }
        }
      },
    };
    new Chart(wikidataWorks, wikidataWorksConfig);

    const wikidataCreators = document.getElementById('wikidataCreators');
    const wikidataCreatorsData = {
      labels: [
        'Creators with ACMI IDs',
        'No ACMI IDs',
      ],
      datasets: [{
        label: 'XOS Creators',
        data: [{{ wikidata.creators|length }}, {{ creators }}],
        backgroundColor: [
          'rgb(74, 192, 192)',
          'rgb(255, 99, 131)',
        ],
        borderColor: 'black',
        hoverOffset: 4
      }]
    };
    const wikidataCreatorsConfig = {
      type: 'doughnut',
      data: wikidataCreatorsData,
      options: {
        responsive: true,
        plugins: {
          legend: {
            position: 'bottom',
          },
          title: {
            display: true,
            text: 'Wikidata Creators',
          }
        }
      },
    };
    new Chart(wikidataCreators, wikidataCreatorsConfig);

  </script>
{% endblock %}